# scVelo + PAGA notebook (cleaned)
This notebook is a cleaned, runnable version of your original workflow.

In [ ]:
# === P1: imports, settings, constants, helpers =================================
# This notebook runs a scVelo + PAGA workflow on planarian scRNA-seq:
#  - merge loom files (spliced/unspliced)
#  - attach Seurat-derived celltype labels + fixed Seurat UMAP coordinates
#  - compute PCA/neighbors on a curated gene set (HVGs minus confounders + forced markers)
#  - compute PAGA connectivities per (condition × timepoint), with bootstrap uncertainty
#  - compute ELAC2-specific deltas: (KD - WT) - (GFP - WT)
#  - run RNA velocity and derive a simple potency surrogate from latent time
#  - plot PAGA graphs using a fixed template layout (Seurat UMAP centroids → FA2)

from __future__ import annotations

import os
import re
import math
import gc
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.sparse as sp

import scanpy as sc
import anndata as ad
import scvelo as scv
import loompy

import matplotlib.pyplot as plt

# --- Jupyter-only niceties (safe no-op outside IPython) -----------------------
try:
    import IPython
    if not hasattr(IPython.display, "set_matplotlib_formats"):
        from matplotlib_inline.backend_inline import set_matplotlib_formats as _set_mpl_formats
        IPython.display.set_matplotlib_formats = _set_mpl_formats
except Exception:
    pass

# --- global plotting defaults ------------------------------------------------
sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=300, facecolor="white", ipython_format=["png"])
sc.settings.file_format_figs = "pdf"
scv.settings.set_figure_params(dpi_save=300, format="pdf")

# --- deterministic seed ------------------------------------------------------
RND = 1337
np.random.seed(RND)

# --- folders -----------------------------------------------------------------
CACHE = Path("cache"); CACHE.mkdir(exist_ok=True)
PLOTS = Path("plots"); PLOTS.mkdir(exist_ok=True)
sc.settings.figdir = str(PLOTS)

# --- inputs (EDIT HERE) ------------------------------------------------------
# Seurat-exported metadata with at least: barcode, celltype, UMAP1, UMAP2
META_CSV = "/mnt/d/scRNA-seq/AZ_final_obj/metadata_for_scvelo_final_final.csv"

# Loom files (spliced/unspliced). Keys are prefixes that become part of barcodes.
LOOMS = {
    "wt0_":    "/mnt/d/scRNA-seq/loom_new_anno/W0_possorted_genome_bam_Q0IEG.loom",
    "wt16_":   "/mnt/d/scRNA-seq/loom_new_anno/W16_possorted_genome_bam_HEHGJ.loom",
    "wt24_":   "/mnt/d/scRNA-seq/loom_new_anno/W24_possorted_genome_bam_MUMHX.loom",
    "wt72_":   "/mnt/d/scRNA-seq/loom_new_anno/W72_possorted_genome_bam_3VCHX.loom",
    "gfp0_":   "/mnt/d/scRNA-seq/loom_new_anno/G0_possorted_genome_bam_SPQ1C.loom",
    "gfp16_":  "/mnt/d/scRNA-seq/loom_new_anno/G16_possorted_genome_bam_6NW0I.loom",
    "gfp24_":  "/mnt/d/scRNA-seq/loom_new_anno/G24_possorted_genome_bam_8AAIZ.loom",
    "gfp72_":  "/mnt/d/scRNA-seq/loom_new_anno/G72_possorted_genome_bam_KS8Y2.loom",
    "elac0_":  "/mnt/d/scRNA-seq/loom_new_anno/E0_possorted_genome_bam_J4ERS.loom",
    "elac16_": "/mnt/d/scRNA-seq/loom_new_anno/E16_possorted_genome_bam_3AJSP.loom",
    "elac24_": "/mnt/d/scRNA-seq/loom_new_anno/E24_possorted_genome_bam_KL9NW.loom",
    "elac72_": "/mnt/d/scRNA-seq/loom_new_anno/E72_possorted_genome_bam_NT2B6.loom",
}

TP_ORDER   = ["0hpa", "16hpa", "24hpa", "72hpa"]
COND_ORDER = ["WT", "GFP", "ELAC2 KD"]

# --- preprocessing constants --------------------------------------------------
N_TOP_GENES       = 3000
MIN_SHARED_COUNTS = 10

# --- helpers -----------------------------------------------------------------
def ensure_float32_layers(adx: ad.AnnData, layers=("spliced", "unspliced")) -> None:
    """Force spliced/unspliced layers to float32 to reduce memory use."""
    for L in layers:
        if L in adx.layers and hasattr(adx.layers[L], "dtype") and adx.layers[L].dtype != np.float32:
            adx.layers[L] = adx.layers[L].astype(np.float32)

def _norm_label(x: str) -> str:
    """Normalize labels: unify dash variants, strip extra whitespace."""
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return x
    x = str(x)
    x = re.sub(r"[‐‒–—]", "–", x)           # normalize dashes to en dash
    x = re.sub(r"[\r\t]+", " ", x)
    x = re.sub(r"\s{2,}", " ", x).strip()
    return x

def derive_cond_time_from_barcode(adx: ad.AnnData) -> ad.AnnData:
    """Derive condition + timepoint from barcode prefixes (wt/gfp/elac + 0/16/24/72)."""
    pref = adx.obs_names.to_series().str.extract(r"^(wt|gfp|elac)(0|16|24|72)_", expand=True)
    adx.obs["cond"] = pref[0].map({"wt": "WT", "gfp": "GFP", "elac": "ELAC2 KD"}).astype("category")
    adx.obs["timepoint"] = pref[1].map({"0": "0hpa", "16": "16hpa", "24": "24hpa", "72": "72hpa"}).astype("category")
    adx.obs["cond"] = adx.obs["cond"].cat.set_categories(COND_ORDER, ordered=True)
    adx.obs["timepoint"] = adx.obs["timepoint"].cat.set_categories(TP_ORDER, ordered=True)
    return adx

# --- celltype → family mapping (keep deterministic, no external deps) ---------
import unicodedata

def add_families_inplace(adx: ad.AnnData, source_key: str = "celltype", target_key: str = "celltype_family") -> ad.AnnData:
    """Map detailed celltype labels to broader celltype families."""

    def _canon(x: str) -> str:
        x = "" if x is None else str(x)
        x = x.replace("⁺", "+").replace("–", "-").replace("’", "'")
        x = unicodedata.normalize("NFKD", x)
        x = "".join(ch for ch in x if not unicodedata.combining(ch))
        x = x.lower()
        x = re.sub(r"[\s_\-()/]+", "", x)
        return x

    def family_label(lbl: str) -> str:
        c = _canon(lbl)

        # Neoblast subtypes (specific first)
        if ("σ" in c) and ("neoblast" in c):            return "σ-neoblast"
        if ("ν" in c) and ("neoblast" in c):            return "ν-neoblast"
        if ("ζ" in c) and ("neoblast" in c):            return "ζ-neoblast"
        if ("γ" in c) and ("neoblast" in c):            return "γ-neoblast"
        if ("pharyngeal" in c) and ("neoblast" in c):   return "Pharyngeal neoblast"
        if ("muscle" in c) and ("neoblast" in c):       return "Muscle neoblast"
        if ("protonephrid" in c) and ("neoblast" in c): return "Protonephridial neoblast"
        if ("parenchymal" in c) and ("neoblast" in c):  return "Parenchymal neoblast"
        if ("eye" in c) and ("neoblast" in c):          return "Eye neoblast"

        # Progenitors
        if ("epiderm" in c) and ("progenitor" in c):    return "Epidermal progenitor"
        if ("eye" in c) and ("progenitor" in c):        return "Eye progenitor"
        if ("pharyngeal" in c) and ("progenitor" in c): return "Pharyngeal progenitor"
        if ("protonephrid" in c) and ("progenitor" in c): return "Protonephridial progenitor"
        if ("parenchymal" in c) and ("progenitor" in c):  return "Parenchymal progenitor"
        if ("neural" in c) and ("progenitor" in c):     return "Neural progenitor"
        if ("pigment" in c) and ("progenitor" in c):    return "Pigment progenitor"

        # Pigment
        if "pigment" in c:                              return "Pigment"

        # Broad families
        if any(k in c for k in ["epidermis", "epidermal"]): return "Epidermis"
        if any(k in c for k in ["photoreceptor", "cup"]) or ("eye" in c): return "Eye"
        if any(k in c for k in ["neuron", "glia", "neural"]): return "Nervous system"
        if ("pharyngeal" in c) or ("parapharyngeal" in c): return "Pharynx"
        if any(k in c for k in ["muscle", "bwm", "pole", "pcg", "ecmproducing"]): return "Muscle"
        if "protonephrid" in c:                          return "Protonephridia"

        if any(k in c for k in ["intestine", "intestinal", "goblet", "phagocyte", "basal"]): return "Intestine"
        if any(k in c for k in ["abracada", "abracadacell"]): return "Parenchyma"
        if "parenchymal" in c:                           return "Parenchyma"

        return "other"

    fam = adx.obs[source_key].astype(str).map(family_label)
    adx.obs[target_key] = fam.astype("category")
    return adx

# placeholders populated later (after neighbors are computed)
GLOBAL_ADJ: sp.csr_matrix | None = None
GLOBAL_DIST: sp.csr_matrix | None = None
NEIGH_CONN_KEY: str | None = None
NEIGH_DIST_KEY: str | None = None


In [ ]:
# === P2: memory-safe loom loading ==========================================

RAW = CACHE / "adata_raw_loom_merged.h5ad"

CHUNK_COLS = 25000
LAYER_SPLICED   = "spliced"
LAYER_UNSPLICED = "unspliced"

def _pick_first_key(d, candidates):
    for k in candidates:
        if k in d:
            return k
    raise KeyError(f"None of the keys {candidates} found. Available: {list(d.keys())}")

def _fix_one_bc(bc: str) -> str:
    bc = re.sub(".*:", "", bc)
    bc = re.sub("x$", "", bc)
    bc = f"{bc}-1"
    bc = re.sub(r"-\d+$", "-1", bc)
    return bc

def _loom_genes(ds) -> np.ndarray:
    key = _pick_first_key(ds.ra, ["Gene", "GeneName", "gene", "Accession"])
    g = ds.ra[key].astype(str)
    g = np.array([re.sub(r"\s+", "", x) for x in g])
    return g

def _loom_barcodes(ds) -> np.ndarray:
    key = _pick_first_key(ds.ca, ["CellID", "cellid", "Barcode", "barcode", "obs_names"])
    return ds.ca[key].astype(str)

def _prefixed_barcodes(raw_bcs: np.ndarray, prefix: str) -> np.ndarray:
    fixed = np.array([prefix + _fix_one_bc(bc) for bc in raw_bcs], dtype=object)
    return fixed

def _read_layers(ds, cols: np.ndarray):
    S = ds.layers[LAYER_SPLICED][:, cols]
    U = ds.layers[LAYER_UNSPLICED][:, cols]
    if not sp.issparse(S): S = sp.csc_matrix(S)
    if not sp.issparse(U): U = sp.csc_matrix(U)
    S = S.T.astype(np.float32, copy=False).tocsr()
    U = U.T.astype(np.float32, copy=False).tocsr()
    return S, U

if RAW.exists():
    adata = sc.read_h5ad(RAW)
else:
    meta = pd.read_csv(META_CSV, usecols=["barcode"])
    meta_bcs = (
        meta["barcode"].astype(str).str.strip()
        .str.replace("–", "-", regex=False)
        .tolist()
    )
    meta_set = set(meta_bcs)
    del meta

    tmp_files = []

    for prefix, path in LOOMS.items():
        with loompy.connect(path, mode="r") as ds:
            genes    = _loom_genes(ds)
            raw_bcs  = _loom_barcodes(ds)
            prefixed = _prefixed_barcodes(raw_bcs, prefix)

            keep_mask = np.fromiter((b in meta_set for b in prefixed),
                                    dtype=bool, count=prefixed.size)
            keep_idx = np.where(keep_mask)[0]
            if keep_idx.size == 0:
                continue

            for start in range(0, keep_idx.size, CHUNK_COLS):
                cols = keep_idx[start:start + CHUNK_COLS]
                if cols.size == 0:
                    continue

                S, U = _read_layers(ds, cols)

                obs_names = prefixed[cols]
                var = pd.DataFrame(index=genes)
                obs = pd.DataFrame(index=obs_names)
                obs.index.name = "barcode"
                obs["batch"] = prefix.rstrip("_")

                if S.shape[0] != obs.shape[0] or S.shape[1] != var.shape[0]:
                    raise ValueError(f"Shape mismatch: X={S.shape}, obs={obs.shape[0]}, var={var.shape[0]}")

                ad_chunk = ad.AnnData(
                    X=S,
                    obs=obs,
                    var=var,
                    layers={LAYER_SPLICED: S, LAYER_UNSPLICED: U},
                    dtype=np.float32,
                )
                ad_chunk.var_names_make_unique()

                tmp_path = CACHE / f"tmp_{prefix}{start:07d}.h5ad"
                ad_chunk.write(tmp_path)
                tmp_files.append(tmp_path)

                del S, U, ad_chunk, obs, var
                gc.collect()

            del genes, raw_bcs, prefixed, keep_mask, keep_idx
            gc.collect()

    if len(tmp_files) == 0:
        raise RuntimeError("No cells matched metadata barcodes; check META_CSV vs loom barcodes/prefixes.")

    ad_merged = None
    for i, f in enumerate(tmp_files):
        ad_i = sc.read_h5ad(f, backed=None)
        if ad_merged is None:
            ad_merged = ad_i
        else:
            ad_merged = sc.concat([ad_merged, ad_i],
                                  join="outer", merge="same",
                                  label=None, index_unique=None)
        del ad_i
        gc.collect()

    ad_merged.obs.index.name = "barcode"
    ad_merged.var_names_make_unique()
    assert {LAYER_SPLICED, LAYER_UNSPLICED} <= set(ad_merged.layers.keys())

    ad_merged.write(RAW)

    for f in tmp_files:
        try:
            os.remove(f)
        except Exception:
            pass

    adata = ad_merged
    del ad_merged
    gc.collect()

adata


In [ ]:
# === P3: annotate from Seurat metadata, freeze Seurat UMAP ==================

ANNO = CACHE / "adata_annotated.h5ad"
# --- robust palette handling (drop-in replacement for apply_palette) -------
# Colors (families)
FAMILY_COLORS = {
    # Neoblast subtypes 
    "σ-neoblast":               "#E3E3E3",
    "ν-neoblast":               "#707070",
    "ζ-neoblast":               "#8D8D8D",
    "γ-neoblast":               "#AAAAAA",
    "Muscle neoblast":          "#D5D5D5",
    "Protonephridial neoblast": "#B8B8B8",
    "Parenchymal neoblast":     "#B3B3B3",
    "Pharyngeal neoblast":      "#7F7F7F",
    "Eye neoblast":             "#00D3D3",   

    # Progenitors 
    "Epidermal progenitor":     "#ABCEDF",   # Early epidermal progenitor
    "Eye progenitor":           "#00D3D3",   # Eye progenitor
    "Pharyngeal progenitor":    "#D9D900",   # Pharyngeal progenitor
    "Parenchymal progenitor":   "#E39F55",   # NKX2⁺ parenchymal progenitor
    "Protonephridial progenitor":"#874A68",  # align to protonephridia wine block
    "Neural progenitor":        "#DED5F6",   # Glutamatergic neural progenitor
    "Pigment progenitor":       "#C08A5A",

    # Mature 
    "Epidermis":                "#5685BD",   # Epidermis (broad)
    "Eye":                      "#00D3D3",   # Eye progenitor (cyan block)
    "Intestine":                "#497F46",   # Phagocyte (broad)
    "Pigment":                  "#8E5A3C",   # Body pigment cell
    "Muscle":                   "#CC5C5D",   # ECM-producing muscle
    "Nervous system":           "#9171BF",   # Mechanosensory neuron (representative violet)
    "Parenchyma":               "#F3D38E",   # LDLRR-1⁺ parenchymal cell
    "Pharynx":                  "#FFFF00",   # Pharyngeal epithelium
    "Protonephridia":           "#874A68",   # Protonephridial flame cell

    "other":                    "#BDBDBD",
}
# Colors (detailed cell types)
DETAILED_COLS = {
  # --- Epidermis (blue) ---
  "Early epidermal progenitor"             : "#ABCEDF",
  "Late epidermal progenitor"              : "#6693C4",
  "Epidermis (broad)"                      : "#5685BD",
  "Epidermis (multiciliated)"              : "#3468B0",
  "Epidermal progenitor (multiciliated)"   : "#4576B6",
  "Epidermal secretory gland progenitor"   : "#99BFD8",
  
  # --- Eye (cyan) ---
  "Eye progenitor"                         : "#00D3D3",
  
  # --- Intestine / immune-like (green) ---
  "Basal cell"                             : "#D3E4BA",
  "Goblet cell"                            : "#8DB180",
  "Phagocyte (broad)"                     : "#497F46",
  
  # --- Pigment (brown) ---
  "Body pigment progenitor"                : "#C08A5A",
  "Body pigment cell"                      : "#8E5A3C",
  
  # --- Muscle (red) ---
  "Muscle progenitor"                      : "#F2C7CA",
  "BWM (dorsal midline)"                   : "#E6A3A6",
  "ECM-producing muscle"                   : "#CC5C5D",
  "Posterior pole/PCG muscle"              : "#C1393A",
  
  # --- Neoblasts (grey) ---
  "σ-neoblast (broad-lineage)"             : "#E3E3E3",
  "Muscle neoblast"                        : "#D5D5D5",
  "Protonephridial neoblast"               : "#B8B8B8",
  "Pharyngeal neoblast"                    : "#7F7F7F",
  "γ-neoblast (intestinal-fated)"          : "#AAAAAA",
  "ζ-neoblast (epidermal-fated)"           : "#8D8D8D",
  "ν-neoblast (neural-fated)"              : "#707070",
  "GLIRP-1⁺ parenchymal neoblast"          : "#B3B3B3",
  "PGRN⁺ parenchymal neoblast"             : "#BEBEBE",
  "FER3L-2⁺ parenchymal neoblast"          : "#C7C7C7",
  
  # --- Neural lineage (violet) ---
  "Neural progenitor (broad)"              : "#E9E4FA",
  "Glutamatergic neural progenitor"        : "#DED5F6",
  "Neuropeptidergic neural progenitor"     : "#D3C7F2",
  "Mechanosensory neural progenitor"       : "#C9BAEE",
  "PKD⁺ sensory neural progenitor"         : "#BFADE8",
  "Glia"                                   : "#AF98D5",
  "Brain branch neuron"                    : "#D8CCF3",
  "Catecholaminergic neuron"               : "#CDBFEB",
  "Cholinergic neuron"                     : "#C3B2E4",
  "Glutamatergic neuron"                   : "#A58ACE",
  "Mechanosensory neuron"                  : "#9171BF",
  "Neuropeptidergic neuron"                : "#8764B8",
  "PKD⁺ sensory neuron"                    : "#7349A9",
  "Serotonergic neuron"                    : "#693CA2",
  
  # --- Parenchyma (sand) ---
  "AQP⁺ parenchymal cell"                  : "#F9E29D",
  "LDLRR-1⁺ parenchymal cell"              : "#F3D38E",
  "GLIRP-1⁺ parenchymal progenitor"        : "#EDC281",
  "PSAP⁺ parenchymal progenitor"           : "#F1D6A0",
  "PSAP⁺ parenchymal cell"                 : "#EBB670",
  "PGRN⁺ parenchymal cell"                 : "#EFC57F",
  "FER3L-2⁺ parenchymal progenitor"        : "#E8B567",
  "NKX2⁺ parenchymal progenitor"           : "#E39F55",
  "PTF⁺ head parenchymal progenitor"       : "#E6A860",
  "SSPO⁺ parenchymal progenitor"           : "#E19A51",
  "SSPO⁺ parenchymal cell"                 : "#DD8B42",
  "Abraçada cell"                          : "#FAE8B4",
  
  # --- Pharynx (yellow) ---
  "Pharyngeal epithelium"                  : "#FFFF00",
  "Pharyngeal progenitor"                  : "#D9D900",
  "Pharyngeal phagocytic-type cell"        : "#C9C900",
  
  # --- Protonephridia (wine) ---
  "Protonephridial flame cell"             : "#874A68",
  "Protonephridial tubule cell"            : "#87345F"
}

def _canon_label_for_palette(x: str) -> str:
    """
    Canonicalize labels to match palette keys irrespective of case, plus signs,
    spaces, unicode variants.
    """
    x = str(x)
    x = x.replace("⁺", "+").replace("–", "-").replace("—", "-").replace("’", "'")
    x = re.sub(r"\s+", " ", x).strip()
    return x.lower()

def _canon_palette(pal: dict) -> dict:
    """Return dict keyed by canonical labels, preserving original color values."""
    return {_canon_label_for_palette(k): v for k, v in pal.items()}

def apply_palette(adx: ad.AnnData, key: str, palette: dict,
                  fallback: str = "#808080", strict: bool = True):
    """
    Attach a color list to adx.uns[f"{key}_colors"] matching the categories
    of adx.obs[key].

    - Canonicalizes both palette keys and category labels to avoid mismatches.
    - If strict=True, raises with a list of missing labels so you catch typos early.
    """
    if key not in adx.obs:
        raise KeyError(f"obs[{key}] not found")

    cats   = list(adx.obs[key].astype("category").cat.categories)
    pal_map = _canon_palette(palette)

    colors  = []
    missing = []
    for c in cats:
        cc = _canon_label_for_palette(c)
        col = pal_map.get(cc, None)
        if col is None:
            missing.append(c)
            col = fallback
        colors.append(col)

    if strict and missing:
        raise ValueError(f"Palette missing colors for categories in '{key}': {missing}")

    adx.uns[f"{key}_colors"] = colors
    return missing  # so you can log or inspect if strict=False

def audit_palette_coverage(adx: ad.AnnData, key: str, palette: dict):
    """
    Print a concise audit of palette coverage for debugging.
    """
    if key not in adx.obs:
        raise KeyError(f"obs[{key}] not found")

    cats    = list(adx.obs[key].astype("category").cat.categories)
    pal_map = _canon_palette(palette)
    miss    = [c for c in cats if _canon_label_for_palette(c) not in pal_map]

    if miss:
        print(f"[audit] {key}: {len(miss)}/{len(cats)} categories missing in palette:")
        for c in miss:
            print("   -", c)
    else:
        print(f"[audit] {key}: full coverage for {len(cats)} categories")

if ANNO.exists():
    adata = sc.read_h5ad(ANNO)
else:
    meta = pd.read_csv(META_CSV)
    need = {"barcode","celltype","UMAP1","UMAP2"}
    missing = need - set(meta.columns)
    if missing:
        raise ValueError(f"Metadata missing columns: {missing}")

    meta["barcode"]  = meta["barcode"].astype(str).str.strip().str.replace("–", "-", regex=False)
    meta["celltype"] = meta["celltype"].astype(str).map(_norm_label)
    meta = meta.set_index("barcode")

    overlap = adata.obs.index.intersection(meta.index)
    if len(overlap) == 0:
        raise ValueError("No overlap in barcodes between AnnData and metadata")

    adata = adata[overlap, :].copy()
    adata.obs["celltype"] = meta.loc[overlap, "celltype"].astype("category")

    # >>> Seurat UMAP, used everywhere <<<
    seurat_umap = meta.loc[overlap, ["UMAP1","UMAP2"]].to_numpy()
    adata.obsm["X_umap_seurat"] = seurat_umap
    adata.obsm["X_umap"]        = seurat_umap.copy()
    adata.obsm["X_umap_joint"]  = seurat_umap.copy()  # for your template code later

    derive_cond_time_from_barcode(adata)
    add_families_inplace(adata, "celltype", "celltype_family")

    adata.obs["celltype"]        = adata.obs["celltype"].cat.remove_unused_categories()
    adata.obs["celltype_family"] = adata.obs["celltype_family"].cat.remove_unused_categories()

    # palettes (soft, non-strict)
    apply_palette(adata, "celltype",        DETAILED_COLS, strict=False)
    apply_palette(adata, "celltype_family", FAMILY_COLORS, strict=False)
    audit_palette_coverage(adata, "celltype",        DETAILED_COLS)
    audit_palette_coverage(adata, "celltype_family", FAMILY_COLORS)

    adata.write(ANNO)

adata


In [ ]:
# === P4: global preprocessing (HVG, confounder mask, PCA, neighbors) ======

PP = CACHE / "adata_preprocessed.h5ad"

marker_ids = [
        "SMESG000041758.1","SMESG000006121.1","SMESG000025003.1","SMESG000025013.1","SMESG000054781.1",
        "SMESG000028264.1","SMESG000000720.1","SMESG000077404.1","SMESG000077405.1","SMESG000019274.1",
        "SMESG000005380.1","SMESG000051964.1","SMESG000075230.1","SMESG000001180.1","SMESG000040332.1",
        "SMESG000043036.1","SMESG000052888.1","SMESG000022963.1","SMESG000022668.1","SMESG000062589.1",
        "SMESG000027543.1","SMESG000066594.1","SMESG000001830.1","SMESG000001839.1","SMESG000023117.1",
        "SMESG000072183.1","SMESG000016228.1","SMESG000039906.1","SMESG000081245.1","SMESG000026446.1",
        "SMESG000050057.1","SMESG000065255.1","SMESG000020792.1","SMESG000021346.1","SMESG000062615.1",
        "SMESG000016881.1","SMESG000068903.1","SMESG000026025.1","SMESG000074489.1","SMESG000001561.1",
        "SMESG000031965.1","SMESG000005975.1","SMESG000001687.1","SMESG000045025.1","SMESG000050160.1",
        "SMESG000020910.1","SMESG000031796.1","SMESG000031800.1","SMESG000032489.1","SMESG000035182.1",
        "SMESG000013991.1","SMESG000013992.1","SMESG000049781.1","SMESG000049783.1","SMESG000044930.1",
        "SMESG000077075.1","SMESG000027359.1","SMESG000004222.1","SMESG000052644.1","SMESG000041571.1",
        "SMESG000051002.1","SMESG000000615.1","SMESG000054829.1","SMESG000034317.1","SMESG000002557.1",
        "SMESG000037029.1","SMESG000014725.1","SMESG000014728.1","SMESG000014733.1","SMESG000021248.1",
        "SMESG000037982.1","SMESG000014322.1","SMESG000074928.1","SMESG000069322.1","SMESG000072430.1",
        "SMESG000024191.1","SMESG000066476.1","SMESG000005930.1","SMESG000006550.1","SMESG000081129.1",
        "SMESG000034317.1","SMESG000003513.1","SMESG000036363.1","SMESG000036385.1","SMESG000062915.1",
        "SMESG000076173.1","SMESG000037031.1","SMESG000039559.1","SMESG000014588.1","SMESG000079512.1",
        "SMESG000068287.1","SMESG000004840.1","SMESG000051170.1","SMESG000078256.1","SMESG000065235.1",
        "SMESG000066244.1","SMESG000002774.1","SMESG000081885.1","SMESG000011244.1","SMESG000017305.1",
        "SMESG000027459.1","SMESG000081970.1","SMESG000070309.1","SMESG000002173.1","SMESG000010063.1",
        "SMESG000074761.1","SMESG000036375.1","SMESG000069984.1","SMESG000021009.1","SMESG000026774.1",
        "SMESG000068398.1","SMESG000034472.1","SMESG000051357.1","SMESG000041535.1","SMESG000069017.1",
        "SMESG000051356.1","SMESG000024277.1","SMESG000050934.1","SMESG000050924.1","SMESG000018924.1",
        "SMESG000026791.1","SMESG000030049.1","SMESG000052932.1","SMESG000032477.1","SMESG000061339.1",
        "SMESG000012683.1","SMESG000029269.1","SMESG000029304.1","SMESG000066649.1","SMESG000013430.1",
        "SMESG000008070.1","SMESG000037720.1","SMESG000014268.1","SMESG000067397.1","SMESG000067002.1",
        "SMESG000066980.1","SMESG000005014.1","SMESG000059487.1","SMESG000018492.1","SMESG000018504.1",
        "SMESG000033381.1","SMESG000014723.1","SMESG000081952.1","SMESG000022856.1","SMESG000010916.1",
        "SMESG000001774.1","SMESG000057112.1","SMESG000016568.1","SMESG000043039.1","SMESG000020988.1",
        "SMESG000053732.1","SMESG000080359.1","SMESG000021044.1","SMESG000001598.1","SMESG000001609.1",
        "SMESG000000522.1","SMESG000019737.1","SMESG000067985.1","SMESG000023346.1","SMESG000023352.1",
        "SMESG000028474.1","SMESG000005963.1","SMESG000023353.1","SMESG000027826.1","SMESG000002628.1",
        "SMESG000019973.1","SMESG000019976.1","SMESG000019984.1","SMESG000044960.1","SMESG000019457.1",
        "SMESG000025547.1","SMESG000041532.1","SMESG000042714.1","SMESG000042718.1","SMESG000048406.1",
        "SMESG000029000.1","SMESG000065670.1","SMESG000023052.1","SMESG000004239.1","SMESG000006530.1",
        "SMESG000018258.1","SMESG000069088.1","SMESG000031846.1","SMESG000022958.1","SMESG000022356.1",
        "SMESG000033462.1","SMESG000074524.1","SMESG000044603.1","SMESG000069765.1","SMESG000003437.1",
        "SMESG000061191.1","SMESG000071069.1","SMESG000001687.1","SMESG000005975.1","SMESG000022725.1",
        "SMESG000017121.1","SMESG000062486.1","SMESG000072084.1"
    ]

if PP.exists():
    adata = sc.read_h5ad(PP)
else:
    # Geometry based on spliced counts
    if "spliced" in adata.layers:
        adata.X = adata.layers["spliced"]

    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    # HVGs
    try:
        import skmisc  # noqa: F401
        HVG_FLAVOR = "seurat_v3"
    except Exception:
        HVG_FLAVOR = "seurat"

    sc.pp.highly_variable_genes(
        adata,
        n_top_genes=N_TOP_GENES,
        flavor=HVG_FLAVOR,
        subset=False
    )

    # Confounder genes (DNA replication, ribosome, etc.) – keep as you defined
    pattern = r"(SMESG000046293.1|SMESG000005389.1|SMESG000038798.1|SMESG000060238.1|SMESG000057772.1|SMESG000070153.1|SMESG000036830.1|SMESG000014909.1|SMESG000000804.1|SMESG000006063.1|SMESG000006068.1|SMESG000006083.1|SMESG000018037.1|SMESG000023149.1|SMESG000035458.1|SMESG000077264.1|SMESG000074436.1|SMESG000057110.1|SMESG000028669.1|SMESG000054290.1|SMESG000060055.1|SMESG000059543.1|SMESG000036986.|SMESG000022017.1|SMESG000017173.1|SMESG000017172.1)"
    conf = pd.Series(
        pd.Index(adata.var_names).str.contains(pattern, case=False, regex=True),
        index=adata.var_names
    ).to_numpy()

    # Force-keep marker genes in geometry
    try:
        def _canon_id(x: str):
            x = re.sub(r"\s+", "", str(x))
            return re.sub(r"\.\d+$", "", x).lower()
        canon_map = {_canon_id(v): v for v in adata.var_names}
        must_keep = [canon_map[_canon_id(m)] for m in marker_ids if _canon_id(m) in canon_map]
        must_keep_mask = adata.var_names.isin(must_keep).to_numpy()
    except Exception:
        must_keep_mask = np.zeros(adata.n_vars, dtype=bool)

    hvg_mask  = adata.var["highly_variable"].to_numpy()
    geom_mask = (hvg_mask & ~conf) | must_keep_mask
    adata     = adata[:, geom_mask].copy()

    # PCA / neighbors only – UMAP stays Seurat-based
    sc.tl.pca(
        adata,
        n_comps=50,
        svd_solver="arpack",
        random_state=RND,
        zero_center=False
    )
    sc.pp.neighbors(adata, use_rep="X_pca", n_neighbors=20, n_pcs=30, random_state=RND)

    # Do not overwrite X_umap; keep Seurat embedding
    # Keep joint alias pointing to Seurat UMAP
    if "X_umap_seurat" in adata.obsm:
        adata.obsm["X_umap_joint"] = adata.obsm["X_umap_seurat"].copy()

    apply_palette(adata, "celltype",        DETAILED_COLS, strict=False)
    apply_palette(adata, "celltype_family", FAMILY_COLORS, strict=False)

    adata.write(PP)

# Global adjacency
nbrs = adata.uns.get("neighbors", {})
NEIGH_CONN_KEY = nbrs.get("connectivities_key", "connectivities")
NEIGH_DIST_KEY = nbrs.get("distances_key", "distances")

if NEIGH_CONN_KEY not in adata.obsp or NEIGH_DIST_KEY not in adata.obsp:
    raise KeyError("Neighbor graphs not found in .obsp")

GLOBAL_ADJ  = adata.obsp[NEIGH_CONN_KEY].tocsr()
GLOBAL_DIST = adata.obsp[NEIGH_DIST_KEY].tocsr()



In [ ]:
# === P5: balanced subset for fair plotting =================================

BAL = CACHE / "adata_balanced.h5ad"
def balance_per_group(adx, by=("cond","timepoint","celltype"), max_n=None, seed=RND):
    df = adx.obs[list(by)].copy()
    grp = df.groupby(list(by)).size().rename("n").reset_index()
    if max_n is None:
        max_n = int(grp["n"].median()); max_n = max(100, min(max_n, grp["n"].min()))
    keep_idx = []
    rng = np.random.default_rng(seed)
    for _, row in grp.iterrows():
        mask = np.logical_and.reduce([adx.obs[k].values == row[k] for k in by])
        idx = np.where(mask)[0]
        keep_idx.extend(idx.tolist() if idx.size <= max_n else rng.choice(idx, size=max_n, replace=False).tolist())
    keep_idx = np.array(sorted(keep_idx))
    return adx[keep_idx, :].copy()
if BAL.exists():
    adata_bal = sc.read_h5ad(BAL)
else:
    adata_bal = balance_per_group(adata, by=("cond","timepoint","celltype"), max_n=None, seed=RND)
    adata_bal.obsm["X_umap_joint"] = adata.obsm["X_umap_joint"][adata.obs_names.get_indexer(adata_bal.obs_names), :]
    apply_palette(adata_bal, "celltype",        DETAILED_COLS, strict=False)
    apply_palette(adata_bal, "celltype_family", FAMILY_COLORS, strict=False)
    adata_bal.write(BAL)

adata_bal


In [ ]:
# === P6: PAGA edge tables using global neighbors ==========================
def bootstrap_edge_deltas(boot_tbl: pd.DataFrame):
    """
    From bootstrap edge weights, compute ELAC2-specific delta per replicate,
    then summarize per (src, dst, timepoint):

    returns:
      wide:    per-bootstrap table with columns
               src, dst, timepoint, boot,
               WT, GFP, ELAC2 KD, delta_KD, delta_GFP, delta_ELAC2_specific
      summary: per-edge summary with mean/std/2.5%/97.5% of delta_ELAC2_specific
    """
    if boot_tbl.empty:
        return (pd.DataFrame(), pd.DataFrame())

    wide = boot_tbl.pivot_table(
        index=["src", "dst", "timepoint", "boot"],
        columns="cond",
        values="weight",
        aggfunc="mean",
    ).reset_index()

    # ensure columns present
    for c in COND_ORDER:
        if c not in wide.columns:
            wide[c] = 0.0

    wide["delta_KD"]  = wide["ELAC2 KD"] - wide["WT"]
    wide["delta_GFP"] = wide["GFP"]      - wide["WT"]
    wide["delta_ELAC2_specific"] = wide["delta_KD"] - wide["delta_GFP"]

    def q025(x): return np.quantile(x, 0.025)
    def q975(x): return np.quantile(x, 0.975)

    summary = (
        wide
        .groupby(["src", "dst", "timepoint"])["delta_ELAC2_specific"]
        .agg(mean="mean", std="std", q025=q025, q975=q975)
        .reset_index()
    )
    return wide, summary

def _subset(adx, cond=None, tp=None, keep_ct=None, min_cells_per_cat=10):
    q = np.ones(adx.n_obs, dtype=bool)
    if cond is not None: q &= (adx.obs["cond"].values == cond)
    if tp   is not None: q &= (adx.obs["timepoint"].values == tp)
    sub = adx[q, :].copy()
    if keep_ct is not None: sub = sub[sub.obs["celltype"].isin(keep_ct)].copy()
    sub.obs["celltype"] = sub.obs["celltype"].astype("category").cat.remove_unused_categories()
    cats = sub.obs["celltype"].cat.categories
    keep = [c for c in cats if (sub.obs["celltype"] == c).sum() >= min_cells_per_cat]
    sub = sub[sub.obs["celltype"].isin(keep)].copy()
    sub.obs["celltype"] = sub.obs["celltype"].cat.remove_unused_categories()
    sub.obs["celltype_family"] = sub.obs["celltype_family"].astype("category").cat.remove_unused_categories()
    return sub
    
def set_subset_neighbors_from_global(sub: ad.AnnData):
    idx = adata.obs_names.get_indexer(sub.obs_names)
    sub.obsp[NEIGH_CONN_KEY] = GLOBAL_ADJ[idx[:, None], idx].tocsr()
    sub.obsp[NEIGH_DIST_KEY] = GLOBAL_DIST[idx[:, None], idx].tocsr()
    sub.uns["neighbors"] = dict(adata.uns["neighbors"])
    sub.uns["neighbors"]["connectivities_key"] = NEIGH_CONN_KEY
    sub.uns["neighbors"]["distances_key"]      = NEIGH_DIST_KEY

def paga_connectivities_from_global(adx: ad.AnnData, group="celltype"):
    idx     = adata.obs_names.get_indexer(adx.obs_names)
    sub_adj = GLOBAL_ADJ[idx[:, None], idx].tocsr()
    cats    = list(adx.obs[group].cat.categories)
    cl      = adx.obs[group].to_numpy()
    label_to_idx = {c: np.where(cl == c)[0] for c in cats if (cl == c).sum() > 0}

    coo  = sub_adj.tocoo()
    used = set()
    for i, j, _ in zip(coo.row, coo.col, coo.data):
        if i == j:
            continue
        a, b = (i, j) if i < j else (j, i)
        used.add((a, b))
    total_mass = float(sum(sub_adj[a, b] for (a, b) in used))
    if total_mass <= 0:
        total_mass = 1.0

    rows = []
    for i, ci in enumerate(cats):
        if ci not in label_to_idx:
            continue
        Ii = label_to_idx[ci]
        for j, cj in enumerate(cats[i:], start=i):
            if cj not in label_to_idx:
                continue
            Ij = label_to_idx[cj]
            block = sub_adj[Ii[:, None], Ij]
            w = float(block.sum())
            if i == j:
                diag_w = float(sub_adj.diagonal()[Ii].sum())
                w = max(0.0, w - diag_w)
            rows.append((ci, cj, w))

    df = pd.DataFrame(rows, columns=["src","dst","w_raw"])
    df["weight"] = df["w_raw"] / total_mass
    df = df[df["src"] != df["dst"]].copy()
    return df[["src","dst","weight"]]

def paga_tables_by_subset_global(adx: ad.AnnData, group="celltype"):
    rows = []
    for tp in TP_ORDER:
        for cond in COND_ORDER:
            sub = _subset(adx, cond=cond, tp=tp)
            if sub.n_obs == 0 or sub.obs[group].nunique() < 2:
                continue
            df = paga_connectivities_from_global(sub, group=group)
            for _, r in df.iterrows():
                rows.append((r["src"], r["dst"], cond, tp, float(r["weight"])))
    return pd.DataFrame(rows, columns=["src","dst","cond","timepoint","weight"])

# Edge tables
EDGES_CELLTYPE_CSV = CACHE / "paga_edges_long_celltype.csv"
if EDGES_CELLTYPE_CSV.exists():
    edge_tbl_cell = pd.read_csv(EDGES_CELLTYPE_CSV)
else:
    edge_tbl_cell = paga_tables_by_subset_global(adata, group="celltype")
    edge_tbl_cell.to_csv(EDGES_CELLTYPE_CSV, index=False)

EDGES_FAMILY_CSV = CACHE / "paga_edges_long_family.csv"
if EDGES_FAMILY_CSV.exists():
    edge_tbl_fam = pd.read_csv(EDGES_FAMILY_CSV)
else:
    edge_tbl_fam = paga_tables_by_subset_global(adata, group="celltype_family")
    edge_tbl_fam.to_csv(EDGES_FAMILY_CSV, index=False)


In [ ]:
# === B1: bootstrap PAGA connectivities =====================================
def subset_by_cond_tp(
    adx,
    group: str,
    cond=None,
    tp=None,
    keep_labels=None,
    min_cells_per_cat: int = 10,
):
    """
    Subset AnnData by cond/timepoint, keep only categories with >= min_cells_per_cat,
    and ensure adx.obs[group] is categorical with unused categories removed.
    """
    q = np.ones(adx.n_obs, dtype=bool)

    if cond is not None:
        q &= (adx.obs["cond"].to_numpy() == cond)
    if tp is not None:
        q &= (adx.obs["timepoint"].to_numpy() == tp)

    sub = adx[q, :].copy()

    if keep_labels is not None:
        sub = sub[sub.obs[group].isin(keep_labels)].copy()

    # force categorical and drop unused
    sub.obs[group] = sub.obs[group].astype("category")
    sub.obs[group] = sub.obs[group].cat.remove_unused_categories()

    # enforce min cells per category
    vc = sub.obs[group].value_counts()
    keep = vc[vc >= min_cells_per_cat].index
    sub = sub[sub.obs[group].isin(keep)].copy()

    sub.obs[group] = sub.obs[group].astype("category")
    sub.obs[group] = sub.obs[group].cat.remove_unused_categories()

    return sub

def _bootstrap_sample_cells(sub: ad.AnnData, group: str, frac: float, rng: np.random.Generator):
    labels = sub.obs[group].astype("category")
    cats   = list(labels.cat.categories)
    arr    = labels.to_numpy()

    idx_all = []
    for c in cats:
        mask = (arr == c)
        base_idx = np.where(mask)[0]
        if base_idx.size == 0:
            continue
        n = max(1, int(round(frac * base_idx.size)))
        resampled = rng.choice(base_idx, size=n, replace=True)
        idx_all.append(resampled)

    if len(idx_all) == 0:
        return sub[:0, :].copy()

    idx_all = np.concatenate(idx_all)
    return sub[idx_all, :].copy()

def paga_bootstrap_edges(
    adx,
    group: str = "celltype",
    n_boot: int = 100,
    frac: float = 0.8,
    seed: int = RND,
    min_cells_per_cat: int = 10,
) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    all_rows = []

    for tp in TP_ORDER:
        for cond in COND_ORDER:
            base = subset_by_cond_tp(
                adx, group=group, cond=cond, tp=tp,
                min_cells_per_cat=min_cells_per_cat
            )
            if base.n_obs == 0 or base.obs[group].nunique() < 2:
                continue

            for b in range(n_boot):
                boot_sub = _bootstrap_sample_cells(base, group=group, frac=frac, rng=rng)
                if boot_sub.n_obs == 0 or boot_sub.obs[group].nunique() < 2:
                    continue

                # ensure categorical after resampling
                boot_sub.obs[group] = boot_sub.obs[group].astype("category")
                boot_sub.obs[group] = boot_sub.obs[group].cat.remove_unused_categories()

                df = paga_connectivities_from_global(boot_sub, group=group)
                df["cond"]      = cond
                df["timepoint"] = tp
                df["boot"]      = b
                all_rows.append(df)

    if not all_rows:
        return pd.DataFrame(columns=["src","dst","cond","timepoint","boot","weight"])

    boot_tbl = pd.concat(all_rows, ignore_index=True)
    return boot_tbl[["src","dst","cond","timepoint","boot","weight"]]


def summarize_bootstrap_edges(boot_tbl: pd.DataFrame) -> pd.DataFrame:
    """
    Summaries per edge × condition × timepoint:
      mean, std, q025, q975
    """
    if boot_tbl.empty:
        return pd.DataFrame()

    def q025(x): return np.quantile(x, 0.025)
    def q975(x): return np.quantile(x, 0.975)

    agg = (
        boot_tbl
        .groupby(["src","dst","cond","timepoint"])["weight"]
        .agg(mean="mean", std="std", q025=q025, q975=q975)
        .reset_index()
    )
    return agg

# Run bootstrap for cell types and families (you can reduce n_boot initially)
BOOT_CELL_CSV = CACHE / "paga_bootstrap_edges_celltype.csv"
if BOOT_CELL_CSV.exists():
    boot_cell = pd.read_csv(BOOT_CELL_CSV)
else:
    boot_cell = paga_bootstrap_edges(adata, group="celltype", n_boot=100, frac=0.8, seed=RND)
    boot_cell.to_csv(BOOT_CELL_CSV, index=False)

BOOT_FAM_CSV = CACHE / "paga_bootstrap_edges_family.csv"
if BOOT_FAM_CSV.exists():
    boot_fam = pd.read_csv(BOOT_FAM_CSV)
else:
    boot_fam = paga_bootstrap_edges(adata, group="celltype_family", n_boot=100, frac=0.8, seed=RND)
    boot_fam.to_csv(BOOT_FAM_CSV, index=False)

boot_summary_cell = summarize_bootstrap_edges(boot_cell)
boot_summary_fam  = summarize_bootstrap_edges(boot_fam)


In [ ]:
# === B2: summarize bootstrap and build mean-weight lookup ==================
# lookup: (src, dst, cond, timepoint) -> mean bootstrap weight
boot_mean_cell = (
    boot_summary_cell
    .set_index(["src", "dst", "cond", "timepoint"])["mean"]
)

boot_mean_fam = (
    boot_summary_fam
    .set_index(["src", "dst", "cond", "timepoint"])["mean"]
)


In [ ]:
# === P7: ELAC2-specific deltas =============================================

def compute_elac2_specific_edge_deltas(df_edges: pd.DataFrame):
    pivot = df_edges.pivot_table(index=["src","dst","timepoint"],
                                 columns="cond",
                                 values="weight", aggfunc="mean").reset_index()
    for c in COND_ORDER:
        if c not in pivot.columns:
            pivot[c] = 0.0
    pivot["delta_KD"]  = pivot["ELAC2 KD"] - pivot["WT"]
    pivot["delta_GFP"] = pivot["GFP"]      - pivot["WT"]
    pivot["delta_ELAC2_specific"] = pivot["delta_KD"] - pivot["delta_GFP"]

    long = pivot.melt(
        id_vars=["src","dst","timepoint"],
        value_vars=["WT","GFP","ELAC2 KD","delta_KD","delta_GFP","delta_ELAC2_specific"],
        var_name="metric", value_name="value"
    )
    return pivot, long

# celltype
EDGE_PIVOT_CELL = CACHE / "paga_EDGE_ELAC2_specific_celltype.csv"
EDGE_LONG_CELL  = CACHE / "paga_EDGE_ELAC2_specific_long_celltype.csv"
if EDGE_PIVOT_CELL.exists() and EDGE_LONG_CELL.exists():
    pivot_cell = pd.read_csv(EDGE_PIVOT_CELL)
    long_cell  = pd.read_csv(EDGE_LONG_CELL)
else:
    pivot_cell, long_cell = compute_elac2_specific_edge_deltas(edge_tbl_cell)
    pivot_cell.to_csv(EDGE_PIVOT_CELL, index=False)
    long_cell.to_csv(EDGE_LONG_CELL,   index=False)
# From bootstrap edge tables boot_cell / boot_fam    
boot_wide_cell, boot_delta_cell = bootstrap_edge_deltas(boot_cell)
# Merge bootstrap CI onto deterministic pivot tables
if not boot_delta_cell.empty:
    pivot_cell = pivot_cell.merge(
        boot_delta_cell.rename(
            columns={
                "mean": "delta_ELAC2_boot_mean",
                "std": "delta_ELAC2_boot_std",
                "q025": "delta_ELAC2_boot_q025",
                "q975": "delta_ELAC2_boot_q975",
            }
        ),
        on=["src", "dst", "timepoint"],
        how="left",
    )
    
for tp in TP_ORDER:
    sub = pivot_cell[pivot_cell["timepoint"] == tp].copy()
    sub = sub[sub["src"] != sub["dst"]]
    sub.sort_values("delta_ELAC2_specific", ascending=False).head(30).to_csv(
        CACHE / f"paga_EDGE_ELAC2_specific_TOP_increase_celltype_{tp}.csv", index=False
    )
    sub.sort_values("delta_ELAC2_specific", ascending=True).head(30).to_csv(
        CACHE / f"paga_EDGE_ELAC2_specific_TOP_decrease_celltype_{tp}.csv", index=False
    )

# family
EDGE_PIVOT_FAM = CACHE / "paga_EDGE_ELAC2_specific_family.csv"
EDGE_LONG_FAM  = CACHE / "paga_EDGE_ELAC2_specific_long_family.csv"
if EDGE_PIVOT_FAM.exists() and EDGE_LONG_FAM.exists():
    pivot_fam = pd.read_csv(EDGE_PIVOT_FAM)
    long_fam  = pd.read_csv(EDGE_LONG_FAM)
else:
    pivot_fam, long_fam = compute_elac2_specific_edge_deltas(edge_tbl_fam)
    pivot_fam.to_csv(EDGE_PIVOT_FAM, index=False)
    long_fam.to_csv(EDGE_LONG_FAM,   index=False)


# From bootstrap edge tables boot_cell / boot_fam
boot_wide_fam,  boot_delta_fam  = bootstrap_edge_deltas(boot_fam)

if not boot_delta_fam.empty:
    pivot_fam = pivot_fam.merge(
        boot_delta_fam.rename(
            columns={
                "mean": "delta_ELAC2_boot_mean",
                "std": "delta_ELAC2_boot_std",
                "q025": "delta_ELAC2_boot_q025",
                "q975": "delta_ELAC2_boot_q975",
            }
        ),
        on=["src", "dst", "timepoint"],
        how="left",
    )
    
for tp in TP_ORDER:
    sub = pivot_fam[pivot_fam["timepoint"] == tp].copy()
    sub = sub[sub["src"] != sub["dst"]]
    sub.sort_values("delta_ELAC2_specific", ascending=False).head(30).to_csv(
        CACHE / f"paga_EDGE_ELAC2_specific_TOP_increase_family_{tp}.csv", index=False
    )
    sub.sort_values("delta_ELAC2_specific", ascending=True).head(30).to_csv(
        CACHE / f"paga_EDGE_ELAC2_specific_TOP_decrease_family_{tp}.csv", index=False
    )

pivot_cell.head(10), pivot_fam.head(10)

for tp in TP_ORDER:
    sub = pivot_cell[(pivot_cell["timepoint"] == tp) & (pivot_cell["src"] != pivot_cell["dst"])].copy()

    # robust increases: 2.5% CI > 0
    inc = sub[sub["delta_ELAC2_boot_q025"] > 0].sort_values(
        "delta_ELAC2_specific", ascending=False
    )
    inc.head(30).to_csv(
        CACHE / f"paga_BS_EDGE_ELAC2_specific_TOP_increase_celltype_{tp}.csv",
        index=False,
    )

    # robust decreases: 97.5% CI < 0
    dec = sub[sub["delta_ELAC2_boot_q975"] < 0].sort_values(
        "delta_ELAC2_specific", ascending=True
    )
    dec.head(30).to_csv(
        CACHE / f"paga_BS_EDGE_ELAC2_specific_TOP_decrease_celltype_{tp}.csv",
        index=False,
    )

for tp in TP_ORDER:
    sub = pivot_fam[(pivot_fam["timepoint"] == tp) & (pivot_fam["src"] != pivot_fam["dst"])].copy()

    # robust increases: 2.5% CI > 0
    inc = sub[sub["delta_ELAC2_boot_q025"] > 0].sort_values(
        "delta_ELAC2_specific", ascending=False
    )
    inc.head(30).to_csv(
        CACHE / f"paga_BS_EDGE_ELAC2_specific_TOP_increase_family_{tp}.csv",
        index=False,
    )

    # robust decreases: 97.5% CI < 0
    dec = sub[sub["delta_ELAC2_boot_q975"] < 0].sort_values(
        "delta_ELAC2_specific", ascending=True
    )
    dec.head(30).to_csv(
        CACHE / f"paga_BS_EDGE_ELAC2_specific_TOP_decrease_family_{tp}.csv",
        index=False,
    )


In [ ]:
# === V1_correct: RNA velocity + latent time + potency surrogate ====================

VEL = CACHE / "adata_velocity.h5ad"

if VEL.exists():
    adata_vel = sc.read_h5ad(VEL)
else:
    # Start from the merged object but reset X to raw spliced counts
    adata_vel = adata.copy()
    ensure_float32_layers(adata_vel, layers=("spliced", "unspliced"))

    # Use spliced counts as X for scVelo preprocessing
    adata_vel.X = adata_vel.layers["spliced"].copy()

    # Standard scVelo preprocessing (counts -> normalized/log1p, moments)
    scv.pp.filter_and_normalize(
        adata_vel,
        min_shared_counts=MIN_SHARED_COUNTS,
        n_top_genes=N_TOP_GENES,
        # use default layer handling: operates on X and 'spliced'/'unspliced'
    )
    scv.pp.moments(adata_vel, n_pcs=30, n_neighbors=30)

    # Dynamical model so that latent_time is defined
    # (this is heavier, but what scVelo uses in the Plass 2018-type analyses)
    scv.tl.recover_dynamics(adata_vel)          # you can pass n_jobs=.. if desired
    scv.tl.velocity(adata_vel, mode="dynamical")
    scv.tl.velocity_graph(adata_vel)
    scv.tl.terminal_states(adata_vel)
    scv.tl.velocity_pseudotime(adata_vel)
    scv.tl.latent_time(adata_vel)

    # Make sure UMAP basis is the fixed Seurat UMAP
    idx = adata.obs_names.get_indexer(adata_vel.obs_names)
    adata_vel.obsm["X_umap"] = adata.obsm["X_umap_seurat"][idx, :]

    adata_vel.write(VEL)

# -------------------------------------------------------------------------
# Propagate velocity-based time metrics back to the main AnnData
# -------------------------------------------------------------------------
# join keeps cond/timepoint and celltype_family already present in adata.obs
cols = ["velocity_pseudotime", "latent_time"]

# Drop any stale versions
for c in cols:
    if c in adata.obs:
        del adata.obs[c]

# Overwrite from adata_vel, aligned by index
adata.obs[cols] = adata_vel.obs.loc[adata.obs_names, cols].to_numpy()


# Potency surrogate (Plass-style: early cells high potency, late cells low)
#lt = adata.obs["latent_time"].to_numpy()
#lt_min = np.nanmin(lt)
#lt_max = np.nanmax(lt)
#pot   = 1.0 - (lt - lt_min) / (lt_max - lt_min + 1e-9)
#adata.obs["potency_surrogate"] = pot
# -------------------------------------------------------------------------
# Re-orient potency so σ-neoblast is high-potency
# -------------------------------------------------------------------------
# Ensure numeric latent_time on adata_vel
adata_vel.obs["latent_time"] = pd.to_numeric(
    adata_vel.obs["latent_time"], errors="coerce"
)

lt = adata_vel.obs["latent_time"].to_numpy()
lt_min = np.nanmin(lt)
lt_max = np.nanmax(lt)
t_scaled = (lt - lt_min) / (lt_max - lt_min + 1e-9)   # in [0, 1]

fam = adata_vel.obs["celltype_family"].astype("category")
root_family = "σ-neoblast"

if root_family in list(fam.cat.categories):
    root_mask = (fam == root_family).to_numpy()
    root_mean = np.nanmean(t_scaled[root_mask])
else:
    root_mean = np.nan  # fallback if σ-neoblast not present

# Decide orientation:
# - if σ-neoblast already early (<= 0.5), potency = 1 - t_scaled
# - if σ-neoblast late      (>  0.5), potency = t_scaled
if np.isnan(root_mean) or root_mean <= 0.5:
    cell_pot = 1.0 - t_scaled
else:
    cell_pot = t_scaled

# Store per-cell potency in both velocity and main objects
adata_vel.obs["potency_surrogate"] = cell_pot
adata.obs["potency_surrogate"] = adata_vel.obs.loc[
    adata.obs_names, "potency_surrogate"
].to_numpy()

sc.pl.violin(
    adata,
    keys="potency_surrogate",
    groupby="celltype_family",
    rotation=90
)


family_mean_pot = (
    adata.obs[["celltype_family", "potency_surrogate"]]
    .groupby("celltype_family")["potency_surrogate"]
    .mean()
)

fmin, fmax = family_mean_pot.min(), family_mean_pot.max()
family_pot_norm = (family_mean_pot - fmin) / (fmax - fmin + 1e-9)

adata.obs["family_potency_norm"] = (
    adata.obs["celltype_family"].map(family_pot_norm).astype(float)
)

# -------------------------------------------------------------------------
# Helper: subset velocity object by condition/timepoint and subset graph
# -------------------------------------------------------------------------
def subset_velocity_by_cond_tp(adata_vel, cond=None, tp=None):
    mask = np.ones(adata_vel.n_obs, dtype=bool)
    if cond is not None:
        mask &= (adata_vel.obs["cond"].values == cond)
    if tp is not None:
        mask &= (adata_vel.obs["timepoint"].values == tp)

    sub = adata_vel[mask, :].copy()
    if sub.n_obs == 0:
        return sub

    idx = np.where(mask)[0]

    # subset velocity_graph (and the negative/transition variants if present)
    for key in ["velocity_graph", "velocity_graph_neg", "velocity_graph_diff"]:
        if key in adata_vel.uns:
            G = adata_vel.uns[key]
            if sp.issparse(G):
                sub.uns[key] = G[idx[:, None], idx].tocsr()
            else:
                sub.uns[key] = G[np.ix_(idx, idx)]

    # UMAP basis is already per-cell, slicing handled by AnnData
    return sub

# -------------------------------------------------------------------------
# Velocity plots – per condition & timepoint, using Seurat UMAP as basis
# -------------------------------------------------------------------------
import matplotlib.pyplot as plt

def plot_velocity_grid_per_cond_tp(color_key="celltype",
                                   density=0.7,
                                   arrow_size=1.0,
                                   arrow_length=1.0,
                                   linewidth=0.7):
    for tp in TP_ORDER:
        for cond in COND_ORDER:
            sub = subset_velocity_by_cond_tp(adata_vel, cond=cond, tp=tp)
            if sub.n_obs == 0:
                continue

            # Ensure color key exists
            if color_key not in sub.obs.columns:
                continue

            ax = scv.pl.velocity_embedding_grid(
                sub,
                basis="umap",
                color=color_key,
                density=density,
                arrow_size=arrow_size,
                arrow_length=arrow_length,
                linewidth=linewidth,
                min_mass=1.0,
                show=False,
                legend_loc="right margin",
            )
            fig = ax.get_figure()
            fig.savefig(
                PLOTS / f"velocity_grid_{color_key}_umap_seurat_{cond}_{tp}.pdf",
                dpi=300,
                bbox_inches="tight"
            )
            plt.close(fig)

# Per-condition & timepoint grids, by detailed celltype and by family
plot_velocity_grid_per_cond_tp(color_key="celltype")
plot_velocity_grid_per_cond_tp(color_key="celltype_family")

# -------------------------------------------------------------------------
# Latent time distributions (robustness check: pile-up or shift per cond/tp)
# -------------------------------------------------------------------------
sc.pl.violin(
    adata,
    keys="latent_time",
    groupby="celltype",
    split="timepoint",
    rotation=45
)


In [ ]:
# === P8: template positions + FA2 layout (fixed geometry) ======================
# We build node positions for PAGA plots that are:
#   1) stable across subsets (cond × timepoint), and
#   2) anchored to Seurat UMAP (centroids per category).
#
# Steps:
#   - compute PAGA on the global neighbor graph restricted to a subset
#   - prune edges (top-k per node and/or quantile) and keep an MST backbone
#   - place nodes by ForceAtlas2 starting from Seurat-UMAP centroids

from fa2 import ForceAtlas2
import networkx as nx

# Scanpy expects these keys under .obsp and .uns["neighbors"]
NEIGH_CONN_KEY = "connectivities"
NEIGH_DIST_KEY = "distances"

# Ensure the global graphs are present on adata (persist to disk if you save later)
assert GLOBAL_ADJ is not None and GLOBAL_DIST is not None, "Run P4 (neighbors) first."
assert GLOBAL_ADJ.shape == (adata.n_obs, adata.n_obs)
assert GLOBAL_DIST.shape == (adata.n_obs, adata.n_obs)

adata.obsp[NEIGH_CONN_KEY] = GLOBAL_ADJ.tocsr()
adata.obsp[NEIGH_DIST_KEY] = GLOBAL_DIST.tocsr()

adata.uns.setdefault("neighbors", {})
adata.uns["neighbors"]["connectivities_key"] = NEIGH_CONN_KEY
adata.uns["neighbors"]["distances_key"] = NEIGH_DIST_KEY
adata.uns["neighbors"].setdefault("params", {})
adata.uns["neighbors"]["params"].setdefault("n_neighbors", 20)

def _subset_neighbors_from_global(sub: ad.AnnData) -> None:
    """Attach global neighbor graphs restricted to subset cells."""
    idx = adata.obs_names.get_indexer(sub.obs_names)
    if np.any(idx < 0):
        raise ValueError("Some subset obs_names are not present in the global adata.")

    sub.obsp[NEIGH_CONN_KEY] = adata.obsp[NEIGH_CONN_KEY][idx[:, None], idx].tocsr()
    sub.obsp[NEIGH_DIST_KEY] = adata.obsp[NEIGH_DIST_KEY][idx[:, None], idx].tocsr()
    sub.uns["neighbors"] = {
        "connectivities_key": NEIGH_CONN_KEY,
        "distances_key": NEIGH_DIST_KEY,
        "params": dict(adata.uns.get("neighbors", {}).get("params", {})),
    }
    sub.uns["neighbors"]["params"].setdefault("n_neighbors", adata.uns["neighbors"]["params"].get("n_neighbors", 20))

def _paga_compute(sub: ad.AnnData, groups: str) -> sp.csr_matrix:
    _subset_neighbors_from_global(sub)
    sc.tl.paga(sub, groups=groups)
    C = sub.uns["paga"]["connectivities"].tocsr(copy=True)
    C = C.maximum(C.T)        # enforce symmetry
    C.setdiag(0.0)
    C.eliminate_zeros()
    return C

def _ensure_min_degree(C: sp.csr_matrix, dmin: int) -> sp.csr_matrix:
    """Add strongest missing edges so every node has at least dmin degree."""
    n = C.shape[0]
    C_lil = C.tolil(copy=True)
    full = C.tocsr()
    for i in range(n):
        deg = C_lil.getrow(i).nnz
        if deg >= dmin:
            continue
        neigh = full.getrow(i).toarray().ravel()
        for j in np.argsort(-neigh):
            if j == i:
                continue
            w = float(full[i, j])
            if w <= 0 or C_lil[i, j] != 0:
                continue
            C_lil[i, j] = w
            C_lil[j, i] = w
            deg += 1
            if deg >= dmin:
                break
    C2 = C_lil.tocsr()
    C2.setdiag(0.0)
    C2.eliminate_zeros()
    return C2

def _paga_prune(C: sp.csr_matrix, topk: int | None, q: float | None, keep_mst: bool, dmin: int | None) -> sp.csr_matrix:
    """Prune PAGA adjacency by top-k edges per node and/or global quantile; keep an MST backbone."""
    n = C.shape[0]
    keep = set()

    if topk and topk > 0:
        for i in range(n):
            row = C.getrow(i)
            if row.nnz == 0:
                continue
            idx, dat = row.indices, row.data
            k = min(topk, dat.size)
            top_idx = idx[np.argpartition(dat, -k)[-k:]]
            for j in top_idx:
                keep.add((min(i, j), max(i, j)))

    if q is not None:
        d = C.data
        d = d[d > 0]
        if d.size:
            thr = float(np.quantile(d, q))
            coo = C.tocoo()
            for i, j, w in zip(coo.row, coo.col, coo.data):
                if i != j and w >= thr:
                    keep.add((min(i, j), max(i, j)))

    if keep_mst and C.nnz > 0:
        G = nx.Graph()
        coo = C.tocoo()
        for i, j, w in zip(coo.row, coo.col, coo.data):
            if i != j:
                G.add_edge(i, j, weight=1.0 / (w + 1e-9))
        T = nx.minimum_spanning_tree(G, weight="weight")
        keep |= {(min(u, v), max(u, v)) for u, v in T.edges()}

    rows, cols, vals = [], [], []
    for (i, j) in keep:
        w = float(C[i, j])
        if w > 0:
            rows += [i, j]
            cols += [j, i]
            vals += [w, w]

    C2 = sp.csr_matrix((vals, (rows, cols)), shape=C.shape)
    C2.setdiag(0.0)
    C2.eliminate_zeros()

    if dmin and dmin > 0:
        C2 = _ensure_min_degree(C2, dmin=dmin)

    return C2

def _fa2_layout(C: sp.csr_matrix, seed_xy: np.ndarray, iters=1000, gravity=1.0, scaling_ratio=2.0) -> np.ndarray:
    """ForceAtlas2 layout, initialized from seed positions."""
    fa = ForceAtlas2(
        outboundAttractionDistribution=False,
        linLogMode=False,
        adjustSizes=False,
        edgeWeightInfluence=1.0,
        jitterTolerance=0.2,
        barnesHutOptimize=True,
        barnesHutTheta=1.2,
        scalingRatio=scaling_ratio,
        strongGravityMode=False,
        gravity=gravity,
        verbose=False,
    )
    M = C.tolil().astype(np.float32)
    P = fa.forceatlas2(M, pos=seed_xy.astype(np.float32), iterations=iters)
    return np.asarray(P, dtype=float)

def _umap_centroids(sub: ad.AnnData, groups: str) -> tuple[np.ndarray, list[str]]:
    """Centroids on the (fixed) UMAP for each category."""
    cats = list(sub.obs[groups].astype("category").cat.categories)
    XY = pd.DataFrame(sub.obsm["X_umap"], index=sub.obs_names, columns=["x", "y"])
    cent = XY.join(sub.obs[groups]).groupby(groups)[["x", "y"]].mean().reindex(cats).to_numpy()
    return cent, cats

def _build_template_positions(adx: ad.AnnData, groups: str, topk: int, q: float, dmin: int, keep_mst=True) -> dict[str, np.ndarray]:
    tmp = adx.copy()
    # Ensure the basis is Seurat UMAP, not a re-computed scanpy UMAP
    if "X_umap_joint" in tmp.obsm:
        tmp.obsm["X_umap"] = tmp.obsm["X_umap_joint"].copy()
    C = _paga_compute(tmp, groups)
    Cp = _paga_prune(C, topk=topk, q=q, keep_mst=keep_mst, dmin=dmin)
    seed, cats = _umap_centroids(tmp, groups)
    P = _fa2_layout(Cp, seed_xy=seed, iters=1000, gravity=1.0, scaling_ratio=2.0)
    return {cats[i]: P[i, :] for i in range(len(cats))}

# Build templates (global, reused for all subsets)
DET_TEMPLATE_POS = _build_template_positions(adata, "celltype", topk=3, q=0.60, dmin=2)
FAM_TEMPLATE_POS = _build_template_positions(adata, "celltype_family", topk=2, q=0.40, dmin=1)


In [ ]:
# === P9: PAGA graphs and overlays using Seurat UMAP templates =============
metric_key = "family_potency_norm"
MIN_PAGA_WEIGHT = 0.0  # graph sparsity controlled by _paga_prune, not here

def apply_bootstrap_to_Cpr(
    Cpr: sp.csr_matrix,
    sub: ad.AnnData,
    group: str,
    boot_mean: pd.Series,
    cond: str,
    tp: str,
    min_weight: float | None = None,
) -> sp.csr_matrix:
    """
    Replace PAGA edge weights in Cpr by bootstrap mean weights when available.

    - Cpr:       pruned adjacency (csr, symmetric) in group-category order
    - sub:       subset AnnData (one cond × timepoint)
    - group:     obs column with group labels ('celltype' or 'celltype_family')
    - boot_mean: Series indexed by (src, dst, cond, timepoint) -> mean weight
    - cond, tp:  current condition and timepoint

    We do not drop edges here. Graph sparsity is controlled upstream by
    _paga_prune (topk, q, MST, dmin). Any display thresholding is handled
    only in the plotting helpers via MIN_PAGA_WEIGHT.
    """
    cats = list(sub.obs[group].astype("category").cat.categories)
    if len(cats) == 0:
        return Cpr

    C = Cpr.tolil(copy=True)
    n = len(cats)

    # For each existing edge in C, try to replace with bootstrap mean
    for i in range(n):
        src = cats[i]
        row_idx = C.rows[i]
        row_dat = C.data[i]
        for k, j in enumerate(row_idx):
            if j <= i:
                continue
            if row_dat[k] <= 0:
                continue
            dst = cats[j]

            key     = (src, dst, cond, tp)
            key_sym = (dst, src, cond, tp)

            w_boot = None
            if key in boot_mean.index:
                w_boot = float(boot_mean.loc[key])
            elif key_sym in boot_mean.index:
                w_boot = float(boot_mean.loc[key_sym])

            if w_boot is not None:
                C[i, j] = w_boot
                C[j, i] = w_boot

    C = C.tocsr()
    C.setdiag(0.0)
    C.eliminate_zeros()
    return C


def _recolor_paga_edges(ax, color="0.6", alpha=0.9):
    for coll in ax.collections:
        if isinstance(coll, LineCollection):
            coll.set_color(color)
            coll.set_alpha(alpha)

def _paga_plot_with_template(
    sub: ad.AnnData,
    groups: str,
    template_pos: dict[str, np.ndarray],
    Cpruned: sp.csr_matrix,
    figsize=(18, 14),
    node_size_scale=8,
    edge_width_scale=0.8,
    fontsize=5,
    fname=None,
):
    C_orig = sub.uns["paga"]["connectivities"]

    # Thresholded copy for plotting
    Cplot = Cpruned.copy()
    Cplot.data[Cplot.data < MIN_PAGA_WEIGHT] = 0.0
    Cplot.eliminate_zeros()

    sub.uns["paga"]["connectivities"] = Cplot

    pos = _map_template_pos(sub, groups, template_pos)

    fig, ax = plt.subplots(figsize=figsize)
    sc.pl.paga(
        sub,
        pos=pos,
        frameon=False,
        threshold=0.0,
        node_size_scale=node_size_scale,
        node_size_power=0.6,
        edge_width_scale=edge_width_scale,
        fontsize=fontsize,
        ax=ax,
        show=False,
        save=False,
        title="",
    )
    _recolor_paga_edges(ax, "0.6", 0.9)

    # Edge-confidence legend based on Cplot
    from matplotlib.lines import Line2D

    Ccoo = Cplot.tocoo()
    mask = Ccoo.row < Ccoo.col
    edge_w = Ccoo.data[mask]

    if edge_w.size > 0:
        wmin = float(edge_w.min())
        wmax = float(edge_w.max())
        min_edge_width = 0.2
        max_edge_width = 4.0

        w_low  = wmin
        w_med  = float(np.median(edge_w))
        w_high = wmax
        if np.isclose(w_med, w_high) or np.isclose(w_med, w_low):
            w_med = w_low + 0.5 * (w_high - w_low)

        w_samples = [w_low, w_med, w_high]
        labels = [f"{w_low:.2f} (low)",
                  f"{w_med:.2f} (mid)",
                  f"{w_high:.2f} (high)"]

        def _map_width(w):
            if wmax > wmin:
                base = min_edge_width + (w - wmin) / (wmax - wmin) * (max_edge_width - min_edge_width)
            else:
                base = (min_edge_width + max_edge_width) * 0.5
            return edge_width_scale * base

        legend_elements = [
            Line2D([0], [0], color="0.6", linewidth=_map_width(w), label=lab)
            for w, lab in zip(w_samples, labels)
        ]

        ax.legend(
            handles=legend_elements,
            title="PAGA weight\n(edge confidence)",
            loc="upper right",
            frameon=True,
            fontsize=fontsize,
        )

    if fname:
        fig.savefig(PLOTS / fname, dpi=450, bbox_inches="tight")
        plt.close(fig)

    sub.uns["paga"]["connectivities"] = C_orig

def _paga_plot_metric_with_template(
    sub: ad.AnnData,
    groups: str,
    metric_key: str,
    template_pos: dict[str, np.ndarray],
    Cpruned: sp.csr_matrix,
    figsize=(16, 12),
    node_size_scale=8.0,
    edge_width_scale=0.8,
    fontsize=6,
    fname=None,
    cmap_name="viridis",
    min_edge_width=0.2,
    max_edge_width=4.0,
):
    """
    Custom PAGA plot where nodes are colored by the average of `metric_key`
    per group. Uses precomputed template positions and a pruned adjacency.

    - `groups`      : obs column with cluster labels (e.g. 'celltype')
    - `metric_key`  : obs column with continuous metric (e.g. 'latent_time')
    - `template_pos`: dict {group_label -> (x, y)} from DET/FAM_TEMPLATE_POS
    - `Cpruned`     : csr_matrix of shape (n_groups, n_groups) in category order
    """
    if metric_key not in sub.obs.columns:
        raise KeyError(f"{metric_key} not found in sub.obs")

    # Categories (node order) and template positions
    cats = list(sub.obs[groups].astype("category").cat.categories)
    if len(cats) == 0:
        return

    pos = _map_template_pos(sub, groups, template_pos)  # shape (n_nodes, 2)

    # Mean metric per group (node color)
    metric_mean = (
        sub.obs[[groups, metric_key]]
        .groupby(groups)[metric_key]
        .mean()
        .reindex(cats)
    )
    metric_vals = metric_mean.to_numpy()

    # Node sizes proportional to cell counts in each group
    counts = (
        sub.obs[groups]
        .value_counts()
        .reindex(cats)
        .fillna(0)
        .to_numpy()
    )
    # Avoid zeros to keep nodes visible
    counts = np.maximum(counts, 1.0)
    node_sizes = node_size_scale * np.sqrt(counts)

    # Edge list from pruned adjacency
    Ccoo = Cpruned.tocoo()
    edge_i = []
    edge_j = []
    edge_w = []
    for i, j, w in zip(Ccoo.row, Ccoo.col, Ccoo.data):
        if i >= j:
            continue  # plot each undirected edge once
        edge_i.append(i)
        edge_j.append(j)
        edge_w.append(w)
    edge_w = np.array(edge_w, dtype=float)
    if edge_w.size > 0:
        # Rescale edge weights to a sensible linewidth range
        wmin = edge_w.min()
        wmax = edge_w.max()
        if wmax > wmin:
            edge_widths = min_edge_width + (edge_w - wmin) / (wmax - wmin) * (max_edge_width - min_edge_width)
        else:
            edge_widths = np.full_like(edge_w, (min_edge_width + max_edge_width) * 0.5)
    else:
        edge_widths = np.array([])

    # Plot
    fig, ax = plt.subplots(figsize=figsize)

    # Edges
    for (i, j, lw) in zip(edge_i, edge_j, edge_widths):
        x0, y0 = pos[i, 0], pos[i, 1]
        x1, y1 = pos[j, 0], pos[j, 1]
        ax.plot(
            [x0, x1],
            [y0, y1],
            color="0.6",
            alpha=0.9,
            linewidth=edge_width_scale * lw,
            zorder=1,
        )

    # Nodes (colored by metric)
    cmap = plt.get_cmap(cmap_name)
    sc_nodes = ax.scatter(
        pos[:, 0],
        pos[:, 1],
        c=metric_vals,
        cmap=cmap,
        s=node_sizes,
        edgecolor="black",
        linewidths=0.3,
        zorder=2,
    )

    # Labels
    for k, lbl in enumerate(cats):
        ax.text(
            pos[k, 0],
            pos[k, 1],
            str(lbl),
            fontsize=fontsize,
            ha="center",
            va="center",
            zorder=3,
        )

    # Colorbar
    cbar = plt.colorbar(sc_nodes, ax=ax)
    cbar.set_label(metric_key)

    ax.set_aspect("equal")
    ax.set_axis_off()
    ax.set_title(f"{groups} – {metric_key}", fontsize=fontsize + 1)

    if fname is not None:
        fig.savefig(PLOTS / fname, dpi=450, bbox_inches="tight")
        plt.close(fig)
    else:
        plt.show()

def _paga_overlay_with_template(
    sub: ad.AnnData,
    groups: str,
    template_pos: dict[str, np.ndarray],
    Cpruned: sp.csr_matrix,
    point_size=3,
    point_alpha=0.12,
    figsize=(18, 14),
    node_size_scale=8,
    edge_width_scale=0.8,
    fontsize=5,
    fname=None,
):
    C_orig = sub.uns["paga"]["connectivities"]

    # Use thresholded copy of Cpruned for plotting
    Cplot = Cpruned.copy()
    Cplot.data[Cplot.data < MIN_PAGA_WEIGHT] = 0.0
    Cplot.eliminate_zeros()

    sub.uns["paga"]["connectivities"] = Cplot

    pos = _map_template_pos(sub, groups, template_pos)

    fig, ax = plt.subplots(figsize=figsize)
    sc.pl.umap(
        sub,
        color=groups,
        alpha=point_alpha,
        size=point_size,
        frameon=False,
        show=False,
        title="",
        ax=ax,
    )
    sc.pl.paga(
        sub,
        pos=pos,
        frameon=False,
        threshold=0.0,   # already thresholded via Cplot
        node_size_scale=node_size_scale,
        node_size_power=0.6,
        edge_width_scale=edge_width_scale,
        fontsize=fontsize,
        ax=ax,
        show=False,
        save=False,
        title="",
    )
    _recolor_paga_edges(ax, "0.6", 0.9)

    # Edge-confidence legend based on Cplot
    from matplotlib.lines import Line2D

    Ccoo = Cplot.tocoo()
    mask = Ccoo.row < Ccoo.col
    edge_w = Ccoo.data[mask]

    if edge_w.size > 0:
        wmin = float(edge_w.min())
        wmax = float(edge_w.max())
        min_edge_width = 0.2
        max_edge_width = 4.0

        w_low  = wmin
        w_med  = float(np.median(edge_w))
        w_high = wmax
        if np.isclose(w_med, w_high) or np.isclose(w_med, w_low):
            w_med = w_low + 0.5 * (w_high - w_low)

        w_samples = [w_low, w_med, w_high]
        labels = [f"{w_low:.2f} (low)",
                  f"{w_med:.2f} (mid)",
                  f"{w_high:.2f} (high)"]

        def _map_width(w):
            if wmax > wmin:
                base = min_edge_width + (w - wmin) / (wmax - wmin) * (max_edge_width - min_edge_width)
            else:
                base = (min_edge_width + max_edge_width) * 0.5
            return edge_width_scale * base

        legend_elements = [
            Line2D([0], [0], color="0.6", linewidth=_map_width(w), label=lab)
            for w, lab in zip(w_samples, labels)
        ]

        ax.legend(
            handles=legend_elements,
            title="PAGA weight\n(edge confidence)",
            loc="upper right",
            frameon=True,
            fontsize=fontsize,
        )

    if fname:
        fig.savefig(PLOTS / fname, dpi=450, bbox_inches="tight")
        plt.close(fig)

    sub.uns["paga"]["connectivities"] = C_orig

# -------------------------------------------------------------------------
# Detailed cell types
# -------------------------------------------------------------------------
for tp in TP_ORDER:
    for cond in COND_ORDER:
        sub = _subset(adata, cond=cond, tp=tp)
        if sub.n_obs == 0 or sub.obs["celltype"].nunique() < 2:
            continue

        apply_palette(sub, "celltype", DETAILED_COLS, strict=False)
        _subset_neighbors_from_global(sub)
        sc.tl.paga(sub, groups="celltype")  # undirected, topology only

        C = sub.uns["paga"]["connectivities"].tocsr().maximum(
            sub.uns["paga"]["connectivities"].T
        ).tocsr()
        C.setdiag(0.0)
        C.eliminate_zeros()
        Cpr = _paga_prune(C, topk=3, q=0.60, keep_mst=True, dmin=2)
        # replace PAGA weights by bootstrap mean weights and threshold
        #Cpr = apply_bootstrap_to_Cpr(
        #    Cpr,
        #    sub,
        #    group="celltype",
        #    boot_mean=boot_mean_cell,
        #    cond=cond,
        #    tp=tp,
        #    min_weight=MIN_PAGA_WEIGHT,        
        #)

        # PAGA graph and overlay (cells colored by celltype)
        _paga_plot_with_template(
            sub,
            "celltype",
            DET_TEMPLATE_POS,
            Cpr,
            figsize=(20, 16),
            node_size_scale=20,
            edge_width_scale=0.9,
            fontsize=12,
            fname=f"PAGA_GRAPH_celltype_tpl_top3_q60_{cond}_{tp}.pdf",
        )
        _paga_overlay_with_template(
            sub,
            "celltype",
            DET_TEMPLATE_POS,
            Cpr,
            figsize=(20, 16),
            node_size_scale=20,
            edge_width_scale=0.9,
            fontsize=12,
            fname=f"PAGA_OVERLAY_celltype_tpl_top3_q60_{cond}_{tp}.pdf",
        )

        # PAGA colored by latent time and potency (per node = mean over cells)
        if "latent_time" in sub.obs.columns:
            _paga_plot_metric_with_template(
                sub,
                "celltype",
                "latent_time",
                DET_TEMPLATE_POS,
                Cpr,
                figsize=(20, 16),
                node_size_scale=20,
                edge_width_scale=0.9,
                fontsize=12,
                fname=f"PAGA_LATENTTIME_celltype_tpl_{cond}_{tp}.pdf",
            )
        if "potency_surrogate" in sub.obs.columns:
            _paga_plot_metric_with_template(
                sub,
                "celltype",
                "potency_surrogate",
                DET_TEMPLATE_POS,
                Cpr,
                figsize=(20, 16),
                node_size_scale=20,
                edge_width_scale=0.9,
                fontsize=12,
                fname=f"PAGA_POTENCY_celltype_tpl_{cond}_{tp}.pdf",
            )

# -------------------------------------------------------------------------
# Families
# -------------------------------------------------------------------------
for tp in TP_ORDER:
    for cond in COND_ORDER:
        sub = _subset(adata, cond=cond, tp=tp)
        if sub.n_obs == 0 or sub.obs["celltype_family"].nunique() < 2:
            continue

        apply_palette(sub, "celltype_family", FAMILY_COLORS, strict=False)
        _subset_neighbors_from_global(sub)
        sc.tl.paga(sub, groups="celltype_family")

        C = sub.uns["paga"]["connectivities"].tocsr().maximum(
            sub.uns["paga"]["connectivities"].T
        ).tocsr()
        C.setdiag(0.0)
        C.eliminate_zeros()
        Cpr = _paga_prune(C, topk=2, q=0.40, keep_mst=True, dmin=1)
        #Cpr = apply_bootstrap_to_Cpr(
        #    Cpr,
        #    sub,
        #    group="celltype_family",
        #    boot_mean=boot_mean_fam,
        #    cond=cond,
        #    tp=tp,
        #    min_weight=MIN_PAGA_WEIGHT,
        #)
        _paga_plot_with_template(
            sub,
            "celltype_family",
            FAM_TEMPLATE_POS,
            Cpr,
            figsize=(16, 12),
            node_size_scale=20,
            edge_width_scale=0.8,
            fontsize=12,
            fname=f"PAGA_GRAPH_family_tpl_top2_q40_{cond}_{tp}.pdf",
        )
        _paga_overlay_with_template(
            sub,
            "celltype_family",
            FAM_TEMPLATE_POS,
            Cpr,
            figsize=(16, 12),
            node_size_scale=20,
            edge_width_scale=0.8,
            fontsize=12,
            fname=f"PAGA_OVERLAY_family_tpl_top2_q40_{cond}_{tp}.pdf",
        )

        if "latent_time" in sub.obs.columns:
            _paga_plot_metric_with_template(
                sub,
                "celltype_family",
                "latent_time",
                FAM_TEMPLATE_POS,
                Cpr,
                figsize=(16, 12),
                node_size_scale=20,
                edge_width_scale=0.8,
                fontsize=12,
                fname=f"PAGA_LATENTTIME_family_tpl_{cond}_{tp}.pdf",
            )
        if "potency_surrogate" in sub.obs.columns:
            _paga_plot_metric_with_template(
                sub,
                "celltype_family",
                "potency_surrogate",
                FAM_TEMPLATE_POS,
                Cpr,
                figsize=(16, 12),
                node_size_scale=20,
                edge_width_scale=0.8,
                fontsize=12,
                fname=f"PAGA_POTENCY_family_tpl_{cond}_{tp}.pdf",
            )


In [ ]:
# === P10: concise summary plots (optional) ======================================
# These are quick, publication-style summaries of ELAC2-specific deltas
# derived in P7 (pivot_fam/long_fam and pivot_cell/long_cell).
#
# Outputs go to ./plots as PDF.

import seaborn as sns

def _ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

_ensure_dir(PLOTS)

def plot_family_delta_heatmap(long_fam: pd.DataFrame, tp: str, vlim: float = 0.02) -> None:
    """Heatmap of family-level Δ connectivity (ELAC2-specific) for one timepoint."""
    sub = long_fam[(long_fam["metric"] == "delta_ELAC2_specific") & (long_fam["timepoint"] == tp)].copy()
    if sub.empty:
        return

    labels = sorted(set(sub["src"]) | set(sub["dst"]))
    mat = pd.DataFrame(0.0, index=labels, columns=labels)
    for _, r in sub.iterrows():
        mat.loc[r["src"], r["dst"]] = r["value"]
        mat.loc[r["dst"], r["src"]] = r["value"]

    plt.figure(figsize=(8, 7))
    sns.heatmap(
        mat, cmap="coolwarm", center=0.0,
        vmin=-vlim, vmax=vlim,
        square=True,
        cbar_kws={"label": "Δ connectivity (ELAC2-specific)"},
    )
    plt.title(f"Family-level ELAC2-specific Δ connectivity, {tp}")
    plt.tight_layout()
    plt.savefig(PLOTS / f"PAGA_family_delta_heatmap_{tp}.pdf", dpi=450, bbox_inches="tight")
    plt.close()

def degree_delta(long_tbl: pd.DataFrame, tp: str) -> pd.DataFrame:
    """Undirected degree-like net delta per node (sum of incident Δ edges)."""
    d = long_tbl[(long_tbl["metric"] == "delta_ELAC2_specific") & (long_tbl["timepoint"] == tp)].copy()
    if d.empty:
        return pd.DataFrame(columns=["node", "deg_delta"])
    a = d[["src", "value"]].rename(columns={"src": "node"})
    b = d[["dst", "value"]].rename(columns={"dst": "node"})
    both = pd.concat([a, b], ignore_index=True)
    out = both.groupby("node", as_index=False)["value"].sum().rename(columns={"value": "deg_delta"})
    return out

def plot_family_degree_delta(long_fam: pd.DataFrame, tp: str) -> None:
    df = degree_delta(long_fam, tp).sort_values("deg_delta")
    if df.empty:
        return

    plt.figure(figsize=(6, 5))
    colors = ["#d73027" if v < 0 else "#1a9850" for v in df["deg_delta"]]
    sns.barplot(data=df, x="deg_delta", y="node", palette=colors)
    plt.axvline(0, color="k", linewidth=0.5)
    plt.xlabel("Net Δ connectivity (ELAC2-specific)")
    plt.ylabel("")
    plt.title(f"Net gain/loss of connectivity per family, {tp}")
    plt.tight_layout()
    plt.savefig(PLOTS / f"PAGA_family_net_delta_{tp}.pdf", dpi=450, bbox_inches="tight")
    plt.close()

# Run summary plots for all timepoints
for tp in TP_ORDER:
    plot_family_delta_heatmap(long_fam, tp)
    plot_family_degree_delta(long_fam, tp)
